# Chapter 02: 텐서(Tensor)와 연산

## 학습 목표
- Tensor의 rank(차원), shape, dtype 개념을 정확히 이해한다
- `tf.constant`와 `tf.Variable`의 차이와 사용 시나리오를 파악한다
- 인덱싱, 슬라이싱, reshape 등 텐서 조작 연산을 실습한다
- 행렬 곱, reduce 연산 등 수학 연산을 코드로 구현한다
- NumPy와의 변환 및 상호 운용성을 이해한다

## 목차
1. [수학적 기초: Rank, 행렬 곱, 브로드캐스팅](#수학적-기초)
2. [Tensor란 무엇인가?](#tensor-기초)
3. [tf.constant vs tf.Variable](#constant-vs-variable)
4. [텐서 생성 함수](#텐서-생성)
5. [인덱싱과 슬라이싱](#인덱싱)
6. [형태 변환: reshape, transpose, squeeze](#형태-변환)
7. [수학 연산](#수학-연산)
8. [NumPy 변환](#numpy-변환)
9. [요약](#요약)

## 수학적 기초 <a name='수학적-기초'></a>

### 1. Tensor Rank (텐서 랭크)

텐서의 **rank**는 차원의 수(축의 개수)를 의미합니다:

- **Rank 0 (스칼라):** $x \in \mathbb{R}$ — 예: $x = 5$
- **Rank 1 (벡터):** $\mathbf{v} \in \mathbb{R}^n$ — 예: $\mathbf{v} = [1, 2, 3]$
- **Rank 2 (행렬):** $A \in \mathbb{R}^{m \times n}$ — 예: $2 \times 3$ 행렬
- **Rank 3 이상:** 고차원 텐서 — 예: 이미지 배치 $(N, H, W, C)$

---

### 2. 행렬 곱 (Matrix Multiplication)

행렬 $A \in \mathbb{R}^{m \times k}$와 $B \in \mathbb{R}^{k \times n}$의 곱 $C = AB$:

$$C_{ij} = \sum_{k} A_{ik} B_{kj}$$

결과 행렬 $C \in \mathbb{R}^{m \times n}$의 각 원소 $C_{ij}$는  
$A$의 $i$번째 **행**과 $B$의 $j$번째 **열**의 **내적(dot product)**입니다.

**조건:** $A$의 열 수 = $B$의 행 수 (내부 차원 일치)

---

### 3. 브로드캐스팅 (Broadcasting) 규칙

두 텐서 $A$, $B$의 shape이 다를 때 NumPy/TF는 자동으로 크기를 맞춥니다:

**규칙:** shape을 오른쪽부터 비교하여:
1. 차원 크기가 같으면 그대로 사용
2. 한쪽이 $1$이면 다른 쪽 크기로 **확장(expand)**
3. 한쪽 rank가 낮으면 왼쪽에 $1$을 추가한 후 위 규칙 적용

$$A: (1, 3) + B: (4, 3) \Rightarrow C: (4, 3)$$

$$A: (3, 1) + B: (1, 4) \Rightarrow C: (3, 4)$$

---
### 🐣 초등학생을 위한 수학 개념 친절 설명!

#### 🔢 Rank(랭크)가 뭐예요?

텐서의 **랭크(Rank)**는 숫자를 몇 겹으로 쌓았는지를 나타내요!

| 이름 | 생긴 모습 | 예시 |
|------|-----------|------|
| **Rank 0 (스칼라)** | 그냥 숫자 하나 | `5`, `3.14` |
| **Rank 1 (벡터)** | 숫자가 한 줄로 나란히 | `[1, 2, 3]` |
| **Rank 2 (행렬)** | 숫자가 표(격자)처럼 | 학교 시간표! |
| **Rank 3 이상** | 표를 여러 장 쌓은 것 | RGB 사진 (빨·초·파 3장) |

> 💡 **비유**: Rank 0은 사탕 1개, Rank 1은 사탕 한 줄, Rank 2는 사탕 상자(가로×세로), Rank 3은 사탕 상자를 여러 층 쌓은 것이에요!

---

#### ✖️ 행렬 곱이 뭐예요?

행렬 곱은 특별한 방식으로 두 표를 합치는 연산이에요.

예를 들어, 아래 두 행렬 **A**와 **B**를 곱해 볼게요:

$$A = \begin{pmatrix} 1 & 2 \\ 3 & 4 \end{pmatrix}, \quad B = \begin{pmatrix} 5 & 6 \\ 7 & 8 \end{pmatrix}$$

결과 $C = A \times B$ 의 첫 번째 칸 $C_{00}$ 는 이렇게 구해요:

$$C_{00} = (A의\ 0번\ 행) \cdot (B의\ 0번\ 열) = 1\times5 + 2\times7 = 5 + 14 = 19$$

> 💡 **비유**: A의 한 **가로 줄**과 B의 한 **세로 줄**을 서로 짝지어 곱하고 모두 더해요!

**중요한 조건 ⚠️**: A의 **열(가로 칸)** 수와 B의 **행(세로 칸)** 수가 같아야 해요!

---

#### 📡 브로드캐스팅이 뭐예요?

크기가 다른 두 텐서를 더할 때, 작은 쪽을 **자동으로 복사해서 크기를 맞춰**주는 마법 같은 기능이에요!

예시: `[10, 20, 30]` 한 줄짜리를 4줄짜리 표에 더하면, 그 한 줄이 4번 복사되어요!

```
[10, 20, 30]       →    [10, 20, 30]
               +        [10, 20, 30]   =  각 행에 더하기!
                        [10, 20, 30]
                        [10, 20, 30]
```

> 💡 **비유**: 선생님이 반 전체에게 같은 공지를 복사해서 나눠주는 것과 같아요! 🗒️

---
### 📝 수학 연습 문제 (직접 풀어보세요!)

#### 문제 1: 랭크(Rank) 알아맞히기
아래 텐서들의 Rank가 몇인지 맞혀보세요!

```
(가) 7                          → Rank = ?
(나) [3, 1, 4, 1, 5]            → Rank = ?
(다) [[1, 2], [3, 4], [5, 6]]   → Rank = ?
(라) [[[1,2],[3,4]],[[5,6],[7,8]]] → Rank = ?
```

<details>
<summary>💡 정답 확인</summary>

- (가) Rank **0** — 숫자 하나 (스칼라)
- (나) Rank **1** — 숫자 5개가 한 줄 (벡터)
- (다) Rank **2** — 3행 2열 표 (행렬)
- (라) Rank **3** — 표 두 장을 쌓은 것

</details>

---

#### 문제 2: 행렬 곱 직접 계산하기

아래 행렬 곱의 결과를 손으로 계산해 보세요!

$$A = \begin{pmatrix} 2 & 0 \\ 1 & 3 \end{pmatrix}, \quad B = \begin{pmatrix} 1 & 4 \\ 2 & 1 \end{pmatrix}$$

```
C[0][0] = 2×1 + 0×2 = ?
C[0][1] = 2×4 + 0×1 = ?
C[1][0] = 1×1 + 3×2 = ?
C[1][1] = 1×4 + 3×1 = ?
```

<details>
<summary>💡 정답 확인</summary>

$$C = \begin{pmatrix} 2 & 8 \\ 7 & 7 \end{pmatrix}$$

C[0][0] = 2, C[0][1] = 8, C[1][0] = 7, C[1][1] = 7
</details>

---

#### 문제 3: 브로드캐스팅 결과 예측하기

```
A = [[1, 2, 3]]       shape: (1, 3)
B = [[10],            shape: (3, 1)
     [20],
     [30]]
```

`A + B` 의 결과 shape과 값은 무엇일까요?

<details>
<summary>💡 정답 확인</summary>

결과 shape: **(3, 3)**

```
[[11, 12, 13],
 [21, 22, 23],
 [31, 32, 33]]
```
A가 세로로 3번 복사되고, B가 가로로 3번 복사된 후 더해져요!
</details>

In [ ]:
# 🔬 수학 연습 문제 코드로 직접 확인해보기!

import tensorflow as tf

print("=" * 50)
print("📌 문제 1: 각 텐서의 Rank 확인")
print("=" * 50)
tensors = [
    ("(가) 스칼라",           tf.constant(7)),
    ("(나) 벡터",             tf.constant([3, 1, 4, 1, 5])),
    ("(다) 행렬",             tf.constant([[1,2],[3,4],[5,6]])),
    ("(라) 3D 텐서",          tf.constant([[[1,2],[3,4]],[[5,6],[7,8]]])),
]
for name, t in tensors:
    print(f"  {name}: shape={t.shape}, Rank={tf.rank(t).numpy()}")

print()
print("=" * 50)
print("📌 문제 2: 행렬 곱 검증")
print("=" * 50)
A = tf.constant([[2, 0], [1, 3]], dtype=tf.float32)
B = tf.constant([[1, 4], [2, 1]], dtype=tf.float32)
C = tf.matmul(A, B)
print(f"A =\n{A.numpy()}")
print(f"B =\n{B.numpy()}")
print(f"C = A @ B =\n{C.numpy()}")
print(f"  → C[0][0]={C[0,0].numpy():.0f},  C[0][1]={C[0,1].numpy():.0f}")
print(f"  → C[1][0]={C[1,0].numpy():.0f},  C[1][1]={C[1,1].numpy():.0f}")

print()
print("=" * 50)
print("📌 문제 3: 브로드캐스팅 결과 확인")
print("=" * 50)
A2 = tf.constant([[1, 2, 3]], dtype=tf.float32)           # (1,3)
B2 = tf.constant([[10], [20], [30]], dtype=tf.float32)    # (3,1)
result = A2 + B2
print(f"A shape: {A2.shape}")
print(f"B shape: {B2.shape}")
print(f"결과 shape: {result.shape}")
print(f"결과:\n{result.numpy()}")


In [ ]:
import tensorflow as tf
import numpy as np

print(f"TensorFlow 버전: {tf.__version__}")

## Tensor란 무엇인가? <a name='tensor-기초'></a>

**텐서(Tensor)**는 TensorFlow의 핵심 데이터 구조입니다.  
N차원 배열로, 딥러닝의 모든 데이터(입력, 가중치, 출력)를 표현합니다.

In [ ]:
# Rank(차원)에 따른 텐서 예시

# Rank 0: 스칼라 (scalar) — 단일 숫자
scalar = tf.constant(42)
print(f"[Rank 0] 스칼라")
print(f"  값: {scalar.numpy()}")
print(f"  rank: {tf.rank(scalar).numpy()}")
print(f"  shape: {scalar.shape}")
print(f"  dtype: {scalar.dtype}\n")

# Rank 1: 벡터 (vector) — 1D 배열
vector = tf.constant([1.0, 2.0, 3.0, 4.0])
print(f"[Rank 1] 벡터")
print(f"  값: {vector.numpy()}")
print(f"  rank: {tf.rank(vector).numpy()}")
print(f"  shape: {vector.shape}")
print(f"  dtype: {vector.dtype}\n")

# Rank 2: 행렬 (matrix) — 2D 배열
matrix = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
print(f"[Rank 2] 행렬")
print(f"  값:\n{matrix.numpy()}")
print(f"  rank: {tf.rank(matrix).numpy()}")
print(f"  shape: {matrix.shape}  (2행 3열)")
print(f"  dtype: {matrix.dtype}\n")

# Rank 3: 3D 텐서 — 예: 이미지 배치에서 하나의 RGB 이미지
tensor_3d = tf.constant([[[1,2],[3,4]], [[5,6],[7,8]], [[9,10],[11,12]]])
print(f"[Rank 3] 3D 텐서")
print(f"  shape: {tensor_3d.shape}  (3, 2, 2)")
print(f"  rank: {tf.rank(tensor_3d).numpy()}")

In [ ]:
# dtype(데이터 타입) 탐색
# TF는 다양한 수치 타입을 지원

dtypes_demo = {
    'float16': tf.constant(3.14, dtype=tf.float16),
    'float32': tf.constant(3.14, dtype=tf.float32),   # 기본 실수형
    'float64': tf.constant(3.14, dtype=tf.float64),
    'int32':   tf.constant(42,   dtype=tf.int32),
    'int64':   tf.constant(42,   dtype=tf.int64),
    'bool':    tf.constant(True, dtype=tf.bool),
    'string':  tf.constant("안녕하세요", dtype=tf.string),
}

print("TensorFlow dtype 목록:")
for name, t in dtypes_demo.items():
    val = t.numpy()
    print(f"  tf.{name:<10} | 값: {str(val):<20} | 메모리: {t.dtype.size if hasattr(t.dtype, 'size') else 'N/A'} bytes")

# dtype 변환 (casting)
x_float = tf.constant([1.7, 2.3, 3.9])
x_int   = tf.cast(x_float, dtype=tf.int32)   # 소수점 버림(truncate)
print(f"\nfloat → int 캐스팅:")
print(f"  원본 (float32): {x_float.numpy()}")
print(f"  변환 (int32):   {x_int.numpy()}  (소수점 버림)")

## tf.constant vs tf.Variable <a name='constant-vs-variable'></a>

TensorFlow에는 두 가지 핵심 텐서 타입이 있습니다:

| 구분 | `tf.constant` | `tf.Variable` |
|------|---------------|---------------|
| 변경 가능 여부 | 불변 (Immutable) | 가변 (Mutable) |
| 사용 목적 | 고정된 데이터 (입력, 하이퍼파라미터) | 학습 가능한 파라미터 (가중치, 편향) |
| 그래디언트 추적 | 기본적으로 추적 안 함 | 자동으로 그래디언트 추적 |
| 메모리 | 일반 메모리 | 장치 메모리 (GPU 포함) |

In [ ]:
# tf.constant: 불변(immutable) 텐서
const_tensor = tf.constant([1.0, 2.0, 3.0])
print("tf.constant:")
print(f"  값: {const_tensor.numpy()}")
print(f"  타입: {type(const_tensor)}")

# 재할당은 새 텐서 생성 (기존 텐서 변경 아님)
const_tensor = const_tensor * 2   # 새 텐서가 생성됨
print(f"  *2 후: {const_tensor.numpy()} (새 텐서 생성됨)")

print()

# tf.Variable: 가변(mutable) 텐서 — 신경망 가중치에 사용
var_tensor = tf.Variable([1.0, 2.0, 3.0], name='weights')
print("tf.Variable:")
print(f"  값: {var_tensor.numpy()}")
print(f"  이름: {var_tensor.name}")
print(f"  학습 가능: {var_tensor.trainable}")

# Variable은 in-place 수정 가능
var_tensor.assign([10.0, 20.0, 30.0])      # 값 교체
print(f"  assign 후: {var_tensor.numpy()}")

var_tensor.assign_add([1.0, 1.0, 1.0])     # 더하기
print(f"  assign_add 후: {var_tensor.numpy()}")

var_tensor.assign_sub([0.5, 0.5, 0.5])     # 빼기
print(f"  assign_sub 후: {var_tensor.numpy()}")

In [ ]:
# 실제 신경망에서의 Variable 사용 예시
# 선형 레이어의 가중치(W)와 편향(b)은 Variable이어야 학습이 가능

# 입력 차원 4, 출력 차원 3인 선형 레이어 파라미터
W = tf.Variable(tf.random.normal([4, 3], stddev=0.1), name='weight')
b = tf.Variable(tf.zeros([3]), name='bias')

print("신경망 파라미터 (Variable):")
print(f"  가중치 W: shape={W.shape}, dtype={W.dtype}")
print(f"  편향  b: shape={b.shape}, dtype={b.dtype}")

# 순전파(Forward Pass): y = Wx + b
x_input = tf.constant([[1.0, 2.0, 3.0, 4.0]])  # 배치 크기 1, 특성 4개
y_output = tf.matmul(x_input, W) + b
print(f"\n순전파 결과: x={x_input.numpy()} → y={y_output.numpy()}")
print(f"출력 shape: {y_output.shape}")

## 텐서 생성 함수 <a name='텐서-생성'></a>

TensorFlow는 다양한 패턴의 텐서를 생성하는 함수를 제공합니다.

In [ ]:
# 기본 생성 함수들

# 0으로 채워진 텐서
zeros = tf.zeros([3, 4])
print(f"tf.zeros([3,4]):\n{zeros.numpy()}\n")

# 1로 채워진 텐서
ones = tf.ones([2, 3])
print(f"tf.ones([2,3]):\n{ones.numpy()}\n")

# 단위 행렬 (항등 행렬) — I: 대각선이 1, 나머지 0
eye = tf.eye(4)
print(f"tf.eye(4) — 4x4 단위행렬:\n{eye.numpy()}\n")

# 특정 값으로 채우기
filled = tf.fill([2, 3], value=7.0)
print(f"tf.fill([2,3], 7.0):\n{filled.numpy()}\n")

# 등간격 숫자 (range)
range_1d = tf.range(0, 10, delta=2)   # 0, 2, 4, 6, 8
print(f"tf.range(0, 10, 2): {range_1d.numpy()}")

In [ ]:
# 난수 생성 함수들 (딥러닝에서 가중치 초기화에 필수)

tf.random.set_seed(42)  # 재현성을 위한 시드 설정

# 정규분포 (평균=0, 표준편차=1)
normal = tf.random.normal([3, 3], mean=0.0, stddev=1.0)
print(f"tf.random.normal([3,3]):\n{normal.numpy().round(3)}\n")

# 균등분포 (0~1 사이)
uniform = tf.random.uniform([3, 3], minval=0.0, maxval=1.0)
print(f"tf.random.uniform([3,3], 0~1):\n{uniform.numpy().round(3)}\n")

# Truncated Normal — 극단값 없는 정규분포 (가중치 초기화에 자주 사용)
trunc = tf.random.truncated_normal([3, 3], mean=0.0, stddev=1.0)
print(f"tf.random.truncated_normal([3,3]) (±2σ 이내 값만):\n{trunc.numpy().round(3)}\n")

# 정수 난수
rand_int = tf.random.uniform([2, 4], minval=0, maxval=10, dtype=tf.int32)
print(f"정수 난수 tf.random.uniform(int32, 0~9):\n{rand_int.numpy()}")

## 인덱싱과 슬라이싱 <a name='인덱싱'></a>

TensorFlow의 인덱싱/슬라이싱은 NumPy와 동일한 문법을 사용합니다.

In [ ]:
# 2D 텐서로 인덱싱/슬라이싱 실습
# 형태: 4행 5열 행렬
mat = tf.constant([
    [11, 12, 13, 14, 15],
    [21, 22, 23, 24, 25],
    [31, 32, 33, 34, 35],
    [41, 42, 43, 44, 45]
], dtype=tf.int32)

print(f"원본 행렬 (shape={mat.shape}):")
print(mat.numpy())

print(f"\n단일 원소 [1, 2] (2행 3열): {mat[1, 2].numpy()}")

# 행 슬라이싱
print(f"\n1번 행 (0-indexed): {mat[1].numpy()}")
print(f"0~1번 행 (상위 2행): \n{mat[:2].numpy()}")
print(f"마지막 2행: \n{mat[-2:].numpy()}")

# 열 슬라이싱
print(f"\n2번 열 전체: {mat[:, 2].numpy()}")
print(f"1~3번 열:\n{mat[:, 1:4].numpy()}")

# 부분 행렬 추출
print(f"\n중앙 2x3 부분:\n{mat[1:3, 1:4].numpy()}")

In [ ]:
# 고급 인덱싱: boolean mask, gather

data = tf.constant([10, 20, 30, 40, 50, 60])

# Boolean 마스크 인덱싱
mask = tf.constant([True, False, True, False, True, False])
masked = tf.boolean_mask(data, mask)
print(f"Boolean 마스크 인덱싱: {masked.numpy()}")

# 조건 기반 마스크
cond_mask = data > 30
print(f"data > 30 마스크: {cond_mask.numpy()}")
print(f"30 초과 값들: {tf.boolean_mask(data, cond_mask).numpy()}")

# gather: 특정 인덱스 값들 수집
indices = tf.constant([0, 2, 5])  # 0번, 2번, 5번 인덱스
gathered = tf.gather(data, indices)
print(f"\ntf.gather 인덱스 [0,2,5]: {gathered.numpy()}")

# where: 조건에 따라 두 텐서에서 선택
a = tf.constant([1, 2, 3, 4, 5])
b = tf.constant([10, 20, 30, 40, 50])
condition = tf.constant([True, False, True, False, True])
result = tf.where(condition, a, b)  # True면 a에서, False면 b에서
print(f"\ntf.where (True→a, False→b): {result.numpy()}")

## 형태 변환: reshape, transpose, squeeze <a name='형태-변환'></a>

딥러닝에서 텐서의 형태를 변환하는 작업은 매우 빈번하게 발생합니다.  
예를 들어, 이미지 데이터를 FC 레이어에 입력하기 위해 flatten 하거나,  
배치 차원을 추가하는 등의 작업이 필요합니다.

In [ ]:
# reshape: 원소 수를 유지하며 shape 변환
# 총 원소 수 = shape 각 차원의 곱 (불변)

original = tf.range(24)   # 0~23, shape=(24,)
print(f"원본 shape: {original.shape}")

# 다양한 reshape
r1 = tf.reshape(original, [6, 4])         # 6행 4열
r2 = tf.reshape(original, [2, 3, 4])      # 2x3x4 3D 텐서
r3 = tf.reshape(original, [2, -1])        # -1: 자동 계산 (2x12)
r4 = tf.reshape(original, [-1])           # 1D로 펼치기 (flatten)

print(f"reshape([6,4]):    {r1.shape}")
print(f"reshape([2,3,4]):  {r2.shape}")
print(f"reshape([2,-1]):   {r3.shape}  (-1 → 12 자동 계산)")
print(f"reshape([-1]):     {r4.shape}  (완전 flatten)")

print(f"\n원소 수 유지 확인: 24 = 6×4={6*4} = 2×3×4={2*3*4}")

In [ ]:
# transpose: 축(axis)의 순서를 바꿈

mat = tf.reshape(tf.range(12, dtype=tf.float32), [3, 4])
print(f"원본 행렬 shape: {mat.shape}")
print(mat.numpy())

# 전치 행렬: (3,4) → (4,3)
mat_T = tf.transpose(mat)   # perm 미지정 시 역순
print(f"\n전치 행렬 shape: {mat_T.shape}")
print(mat_T.numpy())

# 3D 텐서의 축 재배열
# 예: 이미지 데이터 (배치, 높이, 너비, 채널) → (배치, 채널, 높이, 너비)
batch_images = tf.zeros([8, 32, 32, 3])    # NHWC 형식
batch_nchw   = tf.transpose(batch_images, perm=[0, 3, 1, 2])  # NCHW 형식
print(f"\n이미지 형식 변환:")
print(f"  NHWC: {batch_images.shape} → NCHW: {batch_nchw.shape}")

# shape (2,3,4) → transpose(perm=[2,0,1]) → shape (4,2,3)
t3 = tf.zeros([2, 3, 4])
t3_perm = tf.transpose(t3, perm=[2, 0, 1])
print(f"\n(2,3,4) → perm=[2,0,1] → {t3_perm.shape}")

In [ ]:
# squeeze와 expand_dims: 크기 1인 차원 제거/추가

# 문제 상황: 모델이 (배치, 값) 형태를 요구하는데 (값,) 형태로 데이터가 있을 때

vec = tf.constant([1.0, 2.0, 3.0])   # shape: (3,)
print(f"원본 벡터 shape: {vec.shape}")

# expand_dims: 지정 위치에 크기 1인 차원 추가
row_vec = tf.expand_dims(vec, axis=0)   # (3,) → (1,3) — 행 벡터
col_vec = tf.expand_dims(vec, axis=1)   # (3,) → (3,1) — 열 벡터
print(f"expand_dims(axis=0): {row_vec.shape}  (행 벡터)")
print(f"expand_dims(axis=1): {col_vec.shape}  (열 벡터)")
print(f"  열 벡터 값:\n{col_vec.numpy()}")

# squeeze: 크기 1인 차원 제거
with_extra = tf.zeros([1, 5, 1, 3])
squeezed_all = tf.squeeze(with_extra)             # 모든 크기 1 차원 제거
squeezed_ax0 = tf.squeeze(with_extra, axis=0)     # 0번 축만 제거
print(f"\n원본 shape: {with_extra.shape}")
print(f"squeeze(전체): {squeezed_all.shape}")
print(f"squeeze(axis=0): {squeezed_ax0.shape}")

# 실제 사용 예: 배치 차원 추가 (단일 샘플을 배치로 처리할 때)
single_image = tf.zeros([28, 28, 1])              # 단일 MNIST 이미지
batched_image = tf.expand_dims(single_image, 0)   # (28,28,1) → (1,28,28,1)
print(f"\n단일 이미지 → 배치 형태:")
print(f"  {single_image.shape} → {batched_image.shape}")

## 수학 연산 <a name='수학-연산'></a>

TensorFlow는 원소별(element-wise) 연산과 행렬 연산을 모두 지원합니다.

In [ ]:
# 원소별(element-wise) 기본 연산
a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
b = tf.constant([[5.0, 6.0], [7.0, 8.0]])

print("a =\n", a.numpy())
print("b =\n", b.numpy())

print("\n원소별 연산:")
print(f"a + b =\n{tf.add(a, b).numpy()}  (== a + b)")
print(f"a - b =\n{tf.subtract(a, b).numpy()}")
print(f"a * b (원소별 곱) =\n{tf.multiply(a, b).numpy()}")
print(f"a / b =\n{tf.divide(a, b).numpy()}")
print(f"a ** 2 =\n{tf.pow(a, 2).numpy()}")
print(f"sqrt(a) =\n{tf.sqrt(a).numpy().round(4)}")

In [ ]:
# 행렬 곱: C_ij = sum_k(A_ik * B_kj)
# tf.matmul() 또는 @ 연산자 사용

# A: (2,3), B: (3,4) → C: (2,4)
A = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)   # shape (2,3)
B = tf.constant([[1, 0, 0, 1],
                 [0, 1, 0, 1],
                 [0, 0, 1, 1]], dtype=tf.float32)             # shape (3,4)

C = tf.matmul(A, B)   # 또는 A @ B

print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")
print(f"C = A @ B shape: {C.shape}")
print(f"C = A @ B:\n{C.numpy()}")

# 수동으로 C[0,0] 검증: A의 0번 행 · B의 0번 열
c00_manual = A[0,0]*B[0,0] + A[0,1]*B[1,0] + A[0,2]*B[2,0]
print(f"\n수동 검증 C[0,0] = 1×1 + 2×0 + 3×0 = {c00_manual.numpy()}")
print(f"결과 일치: C[0,0] = {C[0,0].numpy()}")

In [ ]:
# 브로드캐스팅 실습
# (1,3) + (4,3) → (4,3)

row = tf.constant([[10, 20, 30]], dtype=tf.float32)  # shape (1,3)
mat = tf.constant([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [10,11,12]], dtype=tf.float32)    # shape (4,3)

result = row + mat  # (1,3) + (4,3) → (4,3)
print(f"row shape: {row.shape}")
print(f"mat shape: {mat.shape}")
print(f"결과 shape: {result.shape}")
print(f"결과 (row가 4번 복사되어 더해짐):\n{result.numpy()}")

# (3,1) + (1,4): 두 축 모두 브로드캐스팅
col = tf.constant([[1], [2], [3]], dtype=tf.float32)   # shape (3,1)
row2 = tf.constant([[10, 20, 30, 40]], dtype=tf.float32) # shape (1,4)
outer = col + row2  # (3,1) + (1,4) → (3,4)
print(f"\ncol: {col.shape}, row2: {row2.shape}")
print(f"outer = col + row2 shape: {outer.shape}")
print(f"결과:\n{outer.numpy()}  (외적 덧셈 — 덧셈 외적)")

In [ ]:
# reduce 연산: 축을 따라 집계
# 딥러닝에서 손실 계산, 배치 평균 등에 필수

data = tf.constant([[1.0, 2.0, 3.0],
                    [4.0, 5.0, 6.0],
                    [7.0, 8.0, 9.0]])

print(f"데이터:\n{data.numpy()}\n")

# 전체 합/평균
print(f"전체 합 (reduce_sum):  {tf.reduce_sum(data).numpy()}")
print(f"전체 평균 (reduce_mean): {tf.reduce_mean(data).numpy()}")

# 행 방향 집계 (axis=0: 행을 따라 열별 집계)
print(f"\n열별 합   (axis=0): {tf.reduce_sum(data, axis=0).numpy()}")
print(f"열별 평균 (axis=0): {tf.reduce_mean(data, axis=0).numpy()}")

# 열 방향 집계 (axis=1: 열을 따라 행별 집계)
print(f"\n행별 합   (axis=1): {tf.reduce_sum(data, axis=1).numpy()}")
print(f"행별 평균 (axis=1): {tf.reduce_mean(data, axis=1).numpy()}")

# keepdims=True: 차원 유지
row_sum = tf.reduce_sum(data, axis=1, keepdims=True)
print(f"\nkeep_dims=True 행별 합: shape={row_sum.shape}")
print(row_sum.numpy())

# 최대/최솟값, argmax
print(f"\n최댓값: {tf.reduce_max(data).numpy()}")
print(f"최솟값: {tf.reduce_min(data).numpy()}")
print(f"각 행의 argmax: {tf.argmax(data, axis=1).numpy()}")

---
### 🐣 초등학생을 위한 수학 연산 친절 설명!

#### ➕ 원소별 연산(Element-wise)이 뭐예요?

두 표에서 **같은 위치에 있는 숫자끼리** 계산하는 거예요!

```
a = [[1, 2],    b = [[5, 6],       a + b = [[1+5, 2+6],    =  [[6, 8],
     [3, 4]]         [7, 8]]                [3+7, 4+8]]          [10, 12]]
```

> 💡 마치 두 사람의 성적표에서 같은 과목 점수를 더하는 것과 같아요!

---

#### ➗ reduce 연산이 뭐예요?

여러 숫자를 **하나(또는 더 작은 개수)로 합치는** 연산이에요.

```
data = [[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]
```

| 연산 | 설명 | 결과 |
|------|------|------|
| `reduce_sum` (전체) | 모든 숫자 더하기 | 1+2+…+9 = **45** |
| `reduce_mean` (전체) | 평균 구하기 | 45÷9 = **5.0** |
| `reduce_sum (axis=0)` | **열별** 합계 | [12, 15, 18] |
| `reduce_sum (axis=1)` | **행별** 합계 | [6, 15, 24] |

> 💡 **axis=0** 은 위에서 아래로(열끼리), **axis=1** 은 옆으로(행끼리) 더해요!

---

#### 🔄 행렬 전치(Transpose)가 뭐예요?

행렬을 **대각선을 기준으로 뒤집는** 연산이에요. 행과 열이 바뀌어요!

```
원본          전치(뒤집기)
[[1, 2, 3],  →  [[1, 4],
 [4, 5, 6]]      [2, 5],
                 [3, 6]]
shape (2,3)  →  shape (3,2)
```

> 💡 **비유**: 세로로 긴 표를 가로로 눕히는 것과 같아요! 📋➡️

---
### 📝 수학 연산 연습 문제 (직접 풀어보세요!)

#### 문제 4: 원소별 연산 결과 예측

```python
a = [[3, 6], [9, 12]]
b = [[1, 2], [3, 4]]
```

다음 결과를 계산해 보세요:
```
(가) a + b  = ?
(나) a * b  = ?   # 원소별 곱 (행렬 곱 아님!)
(다) a / b  = ?
(라) a ** 2 = ?   # a의 각 원소를 제곱
```

<details>
<summary>💡 정답 확인</summary>

```
(가) [[4, 8], [12, 16]]
(나) [[3, 12], [27, 48]]
(다) [[3.0, 3.0], [3.0, 3.0]]
(라) [[9, 36], [81, 144]]
```
</details>

---

#### 문제 5: reduce 연산 결과 예측

```python
data = [[2, 4, 6],
        [8, 10, 12]]
```

```
(가) tf.reduce_sum(data)            = ?
(나) tf.reduce_mean(data)           = ?
(다) tf.reduce_sum(data, axis=0)    = ?  # 각 열의 합
(라) tf.reduce_sum(data, axis=1)    = ?  # 각 행의 합
(마) tf.reduce_max(data)            = ?
```

<details>
<summary>💡 정답 확인</summary>

```
(가) 42          (2+4+6+8+10+12 = 42)
(나) 7.0         (42 ÷ 6 = 7)
(다) [10, 14, 18]
(라) [12, 30]
(마) 12
```
</details>

---

#### 문제 6: 전치 행렬 구하기

아래 행렬의 전치 행렬을 구하고, shape이 어떻게 바뀌는지 써보세요!

$$M = \begin{pmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \end{pmatrix} \quad (shape: 2 \times 3)$$

<details>
<summary>💡 정답 확인</summary>

$$M^T = \begin{pmatrix} 1 & 4 \\ 2 & 5 \\ 3 & 6 \end{pmatrix} \quad (shape: 3 \times 2)$$

행과 열이 서로 바뀌었어요! 행의 수 ↔ 열의 수
</details>

In [ ]:
# 🔬 수학 연산 연습 문제 코드 검증!

import tensorflow as tf

print("=" * 50)
print("📌 문제 4: 원소별 연산 결과")
print("=" * 50)
a = tf.constant([[3, 6], [9, 12]], dtype=tf.float32)
b = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
print(f"  (가) a + b =\n{tf.add(a,b).numpy()}")
print(f"  (나) a * b (원소별) =\n{tf.multiply(a,b).numpy()}")
print(f"  (다) a / b =\n{tf.divide(a,b).numpy()}")
print(f"  (라) a ** 2 =\n{tf.pow(a,2).numpy()}")

print()
print("=" * 50)
print("📌 문제 5: reduce 연산 결과")
print("=" * 50)
data = tf.constant([[2, 4, 6], [8, 10, 12]], dtype=tf.float32)
print(f"  (가) reduce_sum(전체)        = {tf.reduce_sum(data).numpy()}")
print(f"  (나) reduce_mean(전체)       = {tf.reduce_mean(data).numpy()}")
print(f"  (다) reduce_sum(axis=0) 열별 = {tf.reduce_sum(data, axis=0).numpy()}")
print(f"  (라) reduce_sum(axis=1) 행별 = {tf.reduce_sum(data, axis=1).numpy()}")
print(f"  (마) reduce_max             = {tf.reduce_max(data).numpy()}")

print()
print("=" * 50)
print("📌 문제 6: 전치 행렬")
print("=" * 50)
M = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
M_T = tf.transpose(M)
print(f"원본 M (shape {M.shape}):\n{M.numpy()}")
print(f"전치 M^T (shape {M_T.shape}):\n{M_T.numpy()}")


In [ ]:
# 선형대수 연산 (tf.linalg)

A = tf.constant([[3.0, 1.0], [1.0, 4.0]])

# 행렬식 (determinant)
det = tf.linalg.det(A)
print(f"행렬 A:\n{A.numpy()}")
print(f"행렬식 det(A) = {det.numpy():.2f}  (= 3×4 - 1×1 = 11)")

# 역행렬
inv_A = tf.linalg.inv(A)
print(f"\n역행렬 A⁻¹:\n{inv_A.numpy().round(4)}")

# 검증: A × A⁻¹ = I (단위행렬)
identity_check = tf.matmul(A, inv_A)
print(f"A × A⁻¹ (단위행렬 확인):\n{identity_check.numpy().round(6)}")

# 고유값/고유벡터
eigenvalues, eigenvectors = tf.linalg.eig(A)
print(f"\n고유값: {eigenvalues.numpy().real.round(4)}")

## NumPy 변환 <a name='numpy-변환'></a>

TensorFlow와 NumPy는 긴밀하게 통합되어 있습니다.  
두 라이브러리 간의 변환이 매우 간편합니다.

In [ ]:
# NumPy ↔ TensorFlow 변환
import numpy as np

# NumPy → TensorFlow
np_array = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
tf_tensor = tf.constant(np_array)           # 방법 1: tf.constant
tf_tensor2 = tf.convert_to_tensor(np_array) # 방법 2: convert_to_tensor

print(f"NumPy → TF:")
print(f"  NumPy shape: {np_array.shape}, dtype: {np_array.dtype}")
print(f"  TF shape:    {tf_tensor.shape}, dtype: {tf_tensor.dtype}")

# TensorFlow → NumPy
back_to_numpy = tf_tensor.numpy()    # .numpy() 메서드
print(f"\nTF → NumPy: {type(back_to_numpy).__name__}")
print(f"  값: {back_to_numpy}")

# 제로 카피(zero-copy): CPU 메모리에서는 데이터를 복사하지 않음
tf_from_np = tf.constant(np_array)
np_from_tf = np.array(tf_from_np)  # NumPy 연산에 직접 TF 텐서 전달 가능
print(f"\nnp.array(tf_tensor) 결과:\n{np_from_tf}")

# TF 텐서를 NumPy 함수에 직접 사용
result = np.sum(tf_tensor)  # TF 텐서를 NumPy 함수에 바로 전달
print(f"\nnp.sum(tf_tensor) = {result}  (TF 텐서를 NumPy 함수에 직접 사용)")

In [ ]:
# 유용한 텐서 조작 함수 모음

# concat: 텐서 이어 붙이기
t1 = tf.constant([[1, 2], [3, 4]])
t2 = tf.constant([[5, 6], [7, 8]])

c0 = tf.concat([t1, t2], axis=0)  # 행 방향 (세로로) 이어 붙이기
c1 = tf.concat([t1, t2], axis=1)  # 열 방향 (가로로) 이어 붙이기
print(f"t1 shape: {t1.shape}, t2 shape: {t2.shape}")
print(f"concat(axis=0) shape: {c0.shape}\n{c0.numpy()}")
print(f"concat(axis=1) shape: {c1.shape}\n{c1.numpy()}")

# stack: 새 차원으로 쌓기
v1 = tf.constant([1, 2, 3])
v2 = tf.constant([4, 5, 6])
stacked0 = tf.stack([v1, v2], axis=0)  # (2,3)
stacked1 = tf.stack([v1, v2], axis=1)  # (3,2)
print(f"\nstack(axis=0): {stacked0.shape}\n{stacked0.numpy()}")
print(f"stack(axis=1): {stacked1.shape}\n{stacked1.numpy()}")

# split: 텐서 분할
big = tf.constant([[1,2,3,4,5,6]])
parts = tf.split(big, num_or_size_splits=3, axis=1)  # 3등분
print(f"\nsplit (3등분): {[p.numpy() for p in parts]}")

## 요약 <a name='요약'></a>

### 핵심 수식 정리

| 연산 | 수식 | TF 코드 |
|------|------|----------|
| 행렬 곱 | $C_{ij} = \sum_k A_{ik}B_{kj}$ | `tf.matmul(A, B)` 또는 `A @ B` |
| 전치 | $A^T_{ij} = A_{ji}$ | `tf.transpose(A)` |
| 원소 합 | $s = \sum_{i,j} A_{ij}$ | `tf.reduce_sum(A)` |
| 원소 평균 | $\bar{A} = \frac{1}{mn}\sum_{i,j} A_{ij}$ | `tf.reduce_mean(A)` |
| 브로드캐스팅 | $(1,n) + (m,n) \to (m,n)$ | 자동 적용 |

### 핵심 개념 체크리스트
- [ ] Tensor rank = 축(axis)의 개수
- [ ] `tf.constant`: 불변 / `tf.Variable`: 가변 (가중치에 사용)
- [ ] reshape는 총 원소 수를 유지해야 함
- [ ] 브로드캐스팅: 크기 1인 차원은 자동 확장
- [ ] `.numpy()`로 TF 텐서 → NumPy 배열 변환

### 다음 챕터 예고: 03. 자동 미분 (GradientTape)

다음 챕터에서는 딥러닝 학습의 핵심 메커니즘인 **역전파(Backpropagation)**를 학습합니다:
- 편미분 $\frac{\partial f}{\partial x}$의 의미
- 연쇄 법칙 $\frac{\partial L}{\partial w} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial w}$
- `tf.GradientTape`으로 자동 미분 구현
- Keras 없이 선형 회귀 수동 학습